In [1]:
# ch4-2.py
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(module='sklearn*', action='ignore', category=DeprecationWarning)

In [2]:
# 데이터로드 (ch4-2(붓꽃데이터).csv : 데이터 원본 파일)
# encoding : 윈도우즈 환경에서의 한글 처리
# engine : python 3.6에서 한글이 포함된 파일이름 사용
rawData_org = pd.read_csv('./NHIS_OPEN_GJ_2017.csv', encoding='CP949', engine='python')

In [3]:
# 원본데이터의 튜플 수와 컬럼수 출력
rawData_org.shape

(1000000, 34)

In [4]:
# 의사결정트리에 사용할 속성리스트
feature_columns_to_use = ['성별코드', '연령대코드(5세단위)', '신장(5Cm단위)', '체중(5Kg 단위)', '허리둘레', '시력(좌)', '시력(우)',
                    '청력(좌)', '청력(우)', '수축기혈압', '이완기혈압', '식전혈당(공복혈당)', '트리글리세라이드', 'HDL콜레스테롤',
                    'LDL콜레스테롤', '요단백', '혈청크레아티닌', '(혈청지오티)AST', '(혈청지오티)ALT', '감마지티피', '흡연상태', '음주여부']

rawData = rawData_org[feature_columns_to_use]

In [5]:
# 속성값이 NULL인 튜플 제외
rawData = rawData.dropna()

# 인덱스 재설정
rawData.reset_index(inplace=True, drop=True)

# 원본데이터의 튜플 수와 컬럼수 출력
rawData.shape

(990910, 22)

In [20]:
import random as rd  # 샘플링을 위한 random 패키지 임포트

# 1) 샘플링을 통한 수량 축소
# 원본데이터 중 50,000개를 튜플만 샘플링
# len() : 데이터프레임의 크기를 구하는 함수
sample_idx = rd.sample(range(0, len(rawData)), 50000)

# 인덱스 값 정렬(오름차순)
sample_idx.sort()

sample_idx[:10]

[9, 19, 20, 43, 51, 53, 66, 67, 82, 84]

In [21]:
# 샘플링 된 인덱스로 구성된 샘플 데이터프레임 생성
rawData_sample = rawData.loc[sample_idx]

rawData_sample.reset_index(inplace=True, drop=True) #  인덱스 재설정

# 첫 10개의 행만 출력
rawData_sample.head(10)

,성별코드,연령대코드(5세단위),신장(5Cm단위),체중(5Kg 단위),허리둘레,시력(좌),시력(우),청력(좌),청력(우),수축기혈압,...,트리글리세라이드,HDL콜레스테롤,LDL콜레스테롤,요단백,혈청크레아티닌,(혈청지오티)AST,(혈청지오티)ALT,감마지티피,흡연상태,음주여부
0,1,8,170.0,70.0,86.0,1.0,1.0,1.0,1.0,122.0,...,82.0,50.0,123.0,1.0,0.7,43.0,46.0,88.0,1.0,1.0
1,2,12,150.0,50.0,80.0,1.2,1.0,1.0,1.0,120.0,...,61.0,97.0,95.0,1.0,0.6,23.0,21.0,16.0,3.0,1.0
2,2,11,155.0,55.0,80.0,0.5,0.5,1.0,1.0,117.0,...,172.0,48.0,131.0,1.0,0.7,19.0,14.0,19.0,1.0,0.0
3,2,14,145.0,50.0,76.0,0.7,1.0,1.0,1.0,136.0,...,131.0,71.0,133.0,1.0,0.5,18.0,14.0,15.0,1.0,0.0
4,1,10,170.0,70.0,87.0,1.0,1.2,1.0,1.0,128.0,...,173.0,52.0,125.0,2.0,0.9,37.0,51.0,109.0,3.0,1.0
5,2,10,160.0,60.0,80.0,0.7,0.8,1.0,1.0,120.0,...,105.0,62.0,86.0,1.0,0.9,17.0,11.0,13.0,1.0,0.0
6,1,9,170.0,80.0,98.0,1.5,1.2,1.0,1.0,131.0,...,141.0,45.0,102.0,2.0,0.7,19.0,32.0,49.0,3.0,0.0
7,1,7,175.0,75.0,78.0,0.9,0.9,1.0,1.0,108.0,...,114.0,53.0,106.0,1.0,1.2,24.0,27.0,35.0,1.0,1.0
8,1,13,160.0,65.0,84.0,0.7,0.7,1.0,1.0,106.0,...,115.0,45.0,113.0,1.0,1.0,24.0,29.0,31.0,3.0,1.0
9,1,12,160.0,60.0,80.0,1.0,1.0,1.0,1.0,128.0,...,144.0,56.0,190.0,1.0,0.8,44.0,51.0,93.0,3.0,1.0


In [22]:
rawData_sample.shape

(50000, 22)

In [23]:
# nonnumeric 속성을 categrical 데이터로 변환
nonnumeric_columns = ['성별코드','음주여부']

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for feature in nonnumeric_columns:
    rawData_sample[feature] = le.fit_transform(rawData_sample[feature])   

In [24]:
# 소스 데이터프레임에서 분류(classification)을 위한 속성 집합
X = rawData_sample.loc[:, feature_columns_to_use[:-1]]  
y = rawData_sample.loc[:, '음주여부']  # 분류 클래스(class)

from sklearn.model_selection import train_test_split  # 분석모형 선택에 관련된 모듈
    
# 자동으로 데이터셋을 트레이닝셋과 테스트셋으로 분리해주는 함수로
# 트레이닝셋과 데이터셋의 비율을 7:3으로 세팅함
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [25]:
# Scikit-Learn 패키지 : 머신 러닝 교육 및 실무를 위한 패키지로 샘플 데이터셋,
# 다양한 기계학습 기법에 대한 함수 등을 포함하고 있음
from sklearn.ensemble import RandomForestClassifier  # 랜덤포리스트 기법에 관련된 모듈

# DecisionTreeClassifier() : 의사결정트리를 생성하는 함수
random_forest = RandomForestClassifier(n_estimators=20, random_state=0)
random_forest

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [26]:
# fit() : 트레이닝 데이터셋을 대상으로 의사결정트리 학습 진행
random_forest.fit(X_train, y_train)

# tree.predict() 함수를 활용하여 의사결정트리를 대상으로 테스트셋을 예측
y_pred_tr = random_forest.predict(X_test)

In [27]:
# 점수 출력
print("Train Set Score1 : {:.2f}".format(random_forest.score(X_train, y_train)))
print("Test Set Score1 : {:.2f}".format(random_forest.score(X_test, y_test)))

Train Set Score1 : 1.00
Test Set Score1 : 0.71


In [28]:
from sklearn.metrics import accuracy_score  # 분류 정확도(classification accuracy)를 계산하는 모듈

# accuracy_score() 함수를 활용하여 테스트셋의 실제 클래스와 예측된 클래스 간 정확도 측정
print('Accuracy: %.2f' % accuracy_score(y_test, y_pred_tr))

Accuracy: 0.71


In [29]:
# RFE(Recursive Feature Elimination)
from sklearn.feature_selection import RFE

select = RFE(RandomForestClassifier(n_estimators=20, random_state=0), n_features_to_select = 10)
select.fit(X_train, y_train)
select

RFE(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False),
  n_features_to_select=10, step=1, verbose=0)

In [30]:
X_train_RFE = select.transform(X_train)
X_test_RFE = select.transform(X_test)
random_forest.fit(X_train_RFE, y_train)

y_pred_tr = random_forest.predict(X_test_RFE)

In [31]:
# 속성(feature) 별 중요도를 저장하는 데이터프레임 생성 
sel_feature = pd.DataFrame({'중요도' : select.ranking_}, index = feature_columns_to_use[:-1])

# 중요도의 내림차순으로 정렬
sel_feature.sort_values(by='중요도', ascending=False)

,중요도
청력(좌),12
청력(우),11
요단백,10
시력(좌),9
시력(우),8
혈청크레아티닌,7
성별코드,6
체중(5Kg 단위),5
(혈청지오티)AST,4
이완기혈압,3


In [32]:
# 점수 출력
print("Train Set Score1 : {:.2f}".format(random_forest.score(X_train_RFE, y_train)))
print("Test Set Score1 : {:.2f}".format(random_forest.score(X_test_RFE, y_test)))

Train Set Score1 : 1.00
Test Set Score1 : 0.71


In [19]:
# accuracy_score() 함수를 활용하여 테스트셋의 실제 클래스와 예측된 클래스 간 정확도 측정
print('Accuracy: %.2f' % accuracy_score(y_test, y_pred_tr))

Accuracy: 0.71
